## Jupyter notebook example - Classification task
### Example using [MESSIDOR2](https://www.adcis.net/en/third-party/messidor2/) dataset
**Application**: Using RETFound for five-category diabetic retinopathy classification

**Author**: Yukun Zhou

**Date**: 30 Nov 2025

**Performance**:

<table align="left">
<tr>
  <th>Accuracy</th>
  <th>Recall</th>
  <th>F1 Score</th>
  <th>ROC AUC</th>
  <th>PR AUC</th>
</tr>
<tr>
  <td>0.7091</td>
  <td>0.5616</td>
  <td>0.6078</td>
  <td>0.9037</td>
  <td>0.6863</td>
</tr>
</table>


## 1. Install environment
1. Follow [RETFound README](https://github.com/rmaphoh/RETFound) to install environment
2. Restart this Jupyter Notebook
3. Select Kernel retfound

In [ ]:
import sys, torch
from pathlib import Path
import os

PROJECT_ROOT = Path.cwd().resolve()

if PROJECT_ROOT.name == 'examples': PROJECT_ROOT = PROJECT_ROOT.parent
os.chdir(PROJECT_ROOT)

print('Project root:', PROJECT_ROOT)
print("sys.executable:", sys.executable)
print("torch version:", torch.__version__)

## 2. Prepare MESSIDOR2 dataset
1. Download from the [shared data pool](https://github.com/rmaphoh/RETFound/blob/main/BENCHMARK.md).
2. Put the data folder under the project directory, e.g. "RETFound/MESSIDOR2"


## 3. Hyperparameter and path settings
1. Can choose finetune or lp (linear probe)
2. Model selection [info](https://github.com/rmaphoh/RETFound#:~:text=In%20train.sh%2C%20the%20model%20can%20be%20selected%20by%20changing%20the%20hyperparameters%20MODEL%2C%20MODEL_ARCH%2C%20FINETUNE%3A)

In [ ]:
from pathlib import Path
ADAPTATION='finetune'
MODEL='RETFound_dinov2'
MODEL_ARCH='retfound_dinov2'
FINETUNE='RETFound_dinov2_meh'
DATASET='MESSIDOR2'
NUM_CLASS=5
DATA_PATH=PROJECT_ROOT/DATASET
BATCH_SIZE=24
EPOCHS=50
INPUT_SIZE=224
WORLD_SIZE=1
TASK=f"{MODEL_ARCH}_{DATASET}_{ADAPTATION}"
OUTPUT_DIR=PROJECT_ROOT/'output_dir'/TASK
print('DATA_PATH:',DATA_PATH)
print('TASK:',TASK)
print('OUTPUT_DIR:',OUTPUT_DIR)

## 4. Fine-tuning and testing RETFound on MESSIDOR2

In [ ]:
import sys

!{sys.executable} main_finetune.py \
  --model {MODEL} \
  --model_arch {MODEL_ARCH} \
  --finetune {FINETUNE} \
  --savemodel \
  --global_pool \
  --batch_size {BATCH_SIZE} \
  --epochs {EPOCHS} \
  --nb_classes {NUM_CLASS} \
  --data_path {DATA_PATH} \
  --input_size {INPUT_SIZE} \
  --task {TASK} \
  --adaptation {ADAPTATION}

## 5. Evaluation-only

In [ ]:
import sys

CKPT = OUTPUT_DIR / "checkpoint-best.pth"

!{sys.executable} main_finetune.py \
  --model {MODEL} \
  --model_arch {MODEL_ARCH} \
  --finetune {FINETUNE} \
  --savemodel \
  --global_pool \
  --batch_size 128 \
  --nb_classes {NUM_CLASS} \
  --data_path {DATA_PATH} \
  --input_size {INPUT_SIZE} \
  --task {TASK} \
  --adaptation {ADAPTATION} \
  --eval \
  --resume {CKPT}
